In [118]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import string

# Importing Dataset

In [119]:
df_fake = pd.read_csv(r"C:\Users\E87271\Desktop\NLP\Fake News Detection - kaggle\Fake.csv\Fake.csv")
df_true = pd.read_csv(r"C:\Users\E87271\Desktop\NLP\Fake News Detection - kaggle\Fake.csv\True.csv")

In [123]:
df_fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [124]:
df_true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [125]:
df_fake["class"] = 0
df_true["class"] = 1

In [ ]:
# df = pd.concat([df_fake, df_true], axis=0)
# df.reset_index(drop=True, inplace=True)
# df = df.sample(frac=1).reset_index(drop=True)
# df.shape

# from sklearn.model_selection import train_test_split
# x_train, x_test, y_train, y_test = train_test_split(df['text'], df['class'], test_size=0.2, random_state=42)

# train = pd.concat([x_train, y_train], axis=1)
# test = pd.concat([x_test, y_test], axis=1)

# train.to_csv('./data/train.csv', index=False)
# test.to_csv('./data/test.csv', index=False)

In [80]:
df_fake.shape, df_true.shape

((23481, 5), (21417, 5))

In [81]:
df_fake_manual_testing = df_fake.tail(10)
df_true_manual_testing = df_true.tail(10)

In [82]:
for i in range(10):
    df_fake.drop(df_fake.tail(1).index,inplace=True)
    df_true.drop(df_true.tail(1).index,inplace=True)

In [83]:
df_fake.shape, df_true.shape

((23471, 5), (21407, 5))

In [84]:
df_fake_manual_testing['class'] = 0
df_true_manual_testing['class'] = 1

C:\Users\E87271\AppData\Local\Temp\ipykernel_11892\1523065411.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fake_manual_testing['class'] = 0
C:\Users\E87271\AppData\Local\Temp\ipykernel_11892\1523065411.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_true_manual_testing['class'] = 1


In [85]:
df_merge = pd.concat([df_fake, df_true], axis=0)
df_merge.head()

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [86]:
df_merge.columns

Index(['title', 'text', 'subject', 'date', 'class'], dtype='object')

In [87]:
df = df_merge.drop(['title','subject','date'], axis=1)

In [88]:
df.isnull().sum()

text     0
class    0
dtype: int64

In [89]:
df = df.sample(frac=1)
df.head()

,text,class
10115,This is a very big deal. It s very likely that...,0
12345,When not lecturing the Trump Administration fo...,0
22947,Patrick Henningsen 21st Century WireJust as 2...,0
7350,NEW YORK (Reuters) - Some of the most controve...,1
20291,These two Presidents could not be more differe...,0


In [90]:
df.value_counts('class')

class
0    23471
1    21407
Name: count, dtype: int64

In [91]:
df.reset_index(inplace=True)
df.drop(['index'], axis=1, inplace=True)
df.head()

,text,class
0,This is a very big deal. It s very likely that...,0
1,When not lecturing the Trump Administration fo...,0
2,Patrick Henningsen 21st Century WireJust as 2...,0
3,NEW YORK (Reuters) - Some of the most controve...,1
4,These two Presidents could not be more differe...,0


#### Prepocessing the Dataframe

In [92]:
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text) 
    text = re.sub("\\W"," ",text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [93]:
df['text'] = df['text'].apply(wordopt)

#### Train Test Split

In [94]:
X = df['text']
y = df['class']

In [95]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

#### Text to Vectors

In [96]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
xv_train = vectorizer.fit_transform(X_train)
xv_test = vectorizer.transform(X_test)

In [97]:
from sklearn.linear_model import LogisticRegression

LR = LogisticRegression()
LR.fit(xv_train, y_train)

LogisticRegression()

In [98]:
pred_lr = LR.predict(xv_test)

In [99]:
LR.score(xv_test, y_test)

0.9866310160427807

In [100]:
print(classification_report(y_test, pred_lr))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5833
           1       0.99      0.99      0.99      5387

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220



#### Decision Tree Classifier Model

In [102]:
from sklearn.tree import DecisionTreeClassifier

DT = DecisionTreeClassifier()
DT.fit(xv_train, y_train)

DecisionTreeClassifier()

In [103]:
pred_dt = DT.predict(xv_test)

In [104]:
DT.score(xv_test, y_test)

0.9954545454545455

In [105]:
print(classification_report(y_test, pred_dt))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5833
           1       1.00      0.99      1.00      5387

    accuracy                           1.00     11220
   macro avg       1.00      1.00      1.00     11220
weighted avg       1.00      1.00      1.00     11220



#### Gradient Boosting Classifier Model

In [106]:
from sklearn.ensemble import GradientBoostingClassifier

GBC = GradientBoostingClassifier()
GBC.fit(xv_train, y_train)

GradientBoostingClassifier()

In [107]:
pred_gbc = GBC.predict(xv_test)

In [108]:
GBC.score(xv_test, y_test)

0.9944741532976827

In [109]:
print(classification_report(y_test, pred_gbc))

              precision    recall  f1-score   support

           0       1.00      0.99      0.99      5833
           1       0.99      1.00      0.99      5387

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220



#### Random Forest Classifier Model

In [110]:
from sklearn.ensemble import RandomForestClassifier
RF = RandomForestClassifier()
RF.fit(xv_train, y_train)

RandomForestClassifier()

In [112]:
pred_rfc = RF.predict(xv_test)

In [113]:
RF.score(xv_test, y_test)

0.9900178253119429

In [114]:
print(classification_report(y_test, pred_rfc))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5833
           1       0.99      0.99      0.99      5387

    accuracy                           0.99     11220
   macro avg       0.99      0.99      0.99     11220
weighted avg       0.99      0.99      0.99     11220



#### Model Testing

In [115]:
def output_label(n):
    if n == 0:
        return "Fake News"
    elif n == 1:
        return "True News"

def manual_testing(news):
    testing_news = {"text":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test['text'] = new_def_test['text'].apply(wordopt)
    new_xv_test = vectorizer.transform(new_def_test['text'])
    pred_LR = LR.predict(new_xv_test)
    pred_DT = DT.predict(new_xv_test)
    pred_GBC = GBC.predict(new_xv_test)
    pred_RFC = RF.predict(new_xv_test)
    
    print("\n\nLogistic Regression Prediction: {} \nDecision Tree Prediction: {} \nGradient Boosting Prediction: {} \nRandom Forest Prediction: {}".format(output_label(pred_LR[0]),output_label(pred_DT[0]),output_label(pred_GBC[0]),output_label(pred_RFC[0])))

In [117]:
news = str(input())
manual_testing(news)



Logistic Regression Prediction: True News 
Decision Tree Prediction: True News 
Gradient Boosting Prediction: True News 
Random Forest Prediction: True News
